In [15]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import numpy as np
import pandas as pd

In [16]:
# ============== SETUP ==============
PERIOD = 2026

# Define reform
option1_reform = Reform.from_dict(
    {
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.base.excess": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional.benefit_cap": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional.bracket": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional.excess": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us"
)

In [17]:
# Create simulations
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5", reform=option1_reform)


In [18]:
baseline.calculate("social_security", 2025, map_to="household").sum() / 1e9

np.float64(1426.5985670245987)

In [19]:
# Calculate change in net income
baseline_income = baseline.calculate("household_net_income", period=PERIOD)
reformed_income = reformed.calculate("household_net_income", period=PERIOD)
change_in_net_income = reformed_income - baseline_income

In [20]:

# ============== CHANGE IN NET INCOME BY QUINTILE ==============
print("=" * 60)
print("CHANGE IN NET INCOME BY QUINTILE")
print("=" * 60)

quintiles_hh = baseline_income.quintile_rank().values
quintiles_hh = np.where(baseline_income.values < 0, 0, quintiles_hh)

for q in range(1, 6):
    q_mask = quintiles_hh == q
    avg_change = np.average(change_in_net_income.values[q_mask], weights=baseline_income.weights[q_mask])
    print(f"Quintile {q}: ${avg_change:,.2f}")


CHANGE IN NET INCOME BY QUINTILE
Quintile 1: $20.82
Quintile 2: $23.15
Quintile 3: $234.33
Quintile 4: $667.51
Quintile 5: $1,948.83


In [21]:
# ============== HOUSEHOLD LEVEL ANALYSIS ==============
print("\n" + "=" * 60)
print("HOUSEHOLD LEVEL")
print("=" * 60)

# Variables to analyze at household level (add more here)
hh_vars = {
    'household_net_income': baseline_income,
    'taxable_social_security': baseline.calculate("taxable_social_security", PERIOD, map_to="household"),
    'social_security': baseline.calculate("social_security", PERIOD, map_to="household"),
    'adjusted_gross_income': baseline.calculate("adjusted_gross_income", PERIOD, map_to="household"),
    'taxable_ss_magi': baseline.calculate("taxable_ss_magi", PERIOD, map_to="household"),
    'combined_income_for_ss_tax': baseline.calculate("tax_unit_combined_income_for_social_security_taxability", PERIOD, map_to="household"),
    'irs_gross_income': baseline.calculate("irs_gross_income", PERIOD, map_to="household"),
}

# Calculate stats for each quintile
for q in range(1, 6):
    q_mask = quintiles_hh == q
    print(f"\nQuintile {q}:")
    for name, var in hh_vars.items():
        avg = np.average(var.values[q_mask], weights=var.weights[q_mask])
        upper = var.values[q_mask].max()
        min_val = var.values[q_mask].min()
        print(f"  {name:40s} | Avg: ${avg:>12,.2f} | Min: ${min_val:>12,.2f} | Max: ${upper:>12,.2f}")



HOUSEHOLD LEVEL

Quintile 1:
  household_net_income                     | Avg: $   22,234.76 | Min: $        0.00 | Max: $   37,326.14
  taxable_social_security                  | Avg: $      179.80 | Min: $        0.00 | Max: $   28,365.75
  social_security                          | Avg: $    2,181.32 | Min: $        0.00 | Max: $   40,252.45
  adjusted_gross_income                    | Avg: $   17,978.79 | Min: $  -44,071.72 | Max: $  431,418.56
  taxable_ss_magi                          | Avg: $   18,038.18 | Min: $        0.00 | Max: $  433,264.81
  combined_income_for_ss_tax               | Avg: $   19,128.84 | Min: $        0.00 | Max: $  433,264.81
  irs_gross_income                         | Avg: $   18,660.73 | Min: $        0.00 | Max: $  431,418.56

Quintile 2:
  household_net_income                     | Avg: $   49,597.06 | Min: $   37,336.14 | Max: $   61,714.32
  taxable_social_security                  | Avg: $      267.29 | Min: $        0.00 | Max: $   30,793.84
  s

In [22]:
# ============== TAX UNIT LEVEL ANALYSIS ==============
print("\n" + "=" * 60)
print("TAX UNIT LEVEL (household_net_income mapped to tax_unit)")
print("=" * 60)

# Get household income at tax_unit level for quintiles
household_income_tu = baseline.calculate("household_net_income", PERIOD, map_to="tax_unit")
quintiles_tu = household_income_tu.quintile_rank().values
quintiles_tu = np.where(household_income_tu.values < 0, 0, quintiles_tu)

# Variables to analyze at tax_unit level (add more here)
tu_vars = {
    'household_net_income': household_income_tu,
    'taxable_social_security': baseline.calculate("taxable_social_security", map_to = "tax_unit", period = PERIOD),
    'adjusted_gross_income': baseline.calculate("adjusted_gross_income", map_to = "tax_unit", period = PERIOD),
    'combined_income_for_ss_tax': baseline.calculate("tax_unit_combined_income_for_social_security_taxability", map_to = "tax_unit", period = PERIOD),
    'social_security': baseline.calculate("social_security", PERIOD, map_to="tax_unit"),
}

# Calculate stats for each quintile
for q in range(1, 6):
    q_mask = quintiles_tu == q
    print(f"\nQuintile {q}:")
    for name, var in tu_vars.items():
        avg = np.average(var.values[q_mask], weights=var.weights[q_mask])
        upper = var.values[q_mask].max()
        min_val = var.values[q_mask].min()
        print(f"  {name:40s} | Avg: ${avg:>12,.2f} | Min: ${min_val:>12,.2f} | Max: ${upper:>12,.2f}")



TAX UNIT LEVEL (household_net_income mapped to tax_unit)

Quintile 1:
  household_net_income                     | Avg: $   13,643.88 | Min: $        0.00 | Max: $   23,205.08
  taxable_social_security                  | Avg: $       14.49 | Min: $        0.00 | Max: $   28,365.75
  adjusted_gross_income                    | Avg: $    9,888.96 | Min: $ -102,011.45 | Max: $  405,007.47
  combined_income_for_ss_tax               | Avg: $   10,715.21 | Min: $        0.00 | Max: $  405,007.47
  social_security                          | Avg: $    1,047.35 | Min: $        0.00 | Max: $   36,228.04

Quintile 2:
  household_net_income                     | Avg: $   32,249.71 | Min: $   23,215.29 | Max: $   42,642.49
  taxable_social_security                  | Avg: $      269.46 | Min: $        0.00 | Max: $   30,793.84
  adjusted_gross_income                    | Avg: $   26,175.23 | Min: $  -82,629.77 | Max: $  489,757.19
  combined_income_for_ss_tax               | Avg: $   28,231.74 | Mi

In [23]:
# ============== Q1 DETAILED RECORDS (HOUSEHOLD LEVEL) ==============
print("\n" + "=" * 60)
print("Q1 RECORDS WITH TAXABLE SS > 0 (HOUSEHOLD LEVEL)")
print("=" * 60)

q1_mask_hh = quintiles_hh == 1
taxable_ss_hh = hh_vars['taxable_social_security']
positive_ss_mask = taxable_ss_hh.values > 0
combined_mask = q1_mask_hh & positive_ss_mask

df_q1_detail = pd.DataFrame({
    'taxable_ss': taxable_ss_hh.values[combined_mask],
    'total_ss': hh_vars['social_security'].values[combined_mask],
    'agi': hh_vars['adjusted_gross_income'].values[combined_mask],
    'net_income': baseline_income.values[combined_mask],
    'taxable_ss_magi': hh_vars['taxable_ss_magi'].values[combined_mask],
    'combined_income_for_ss_tax': hh_vars['combined_income_for_ss_tax'].values[combined_mask],
    'irs_gross_income': hh_vars['irs_gross_income'].values[combined_mask],
    'weight': taxable_ss_hh.weights[combined_mask]
})

print(f"Found {combined_mask.sum()} households in Q1 with taxable SS > 0")
print(df_q1_detail.sort_values('taxable_ss', ascending=False).head(20))



Q1 RECORDS WITH TAXABLE SS > 0 (HOUSEHOLD LEVEL)
Found 17 households in Q1 with taxable SS > 0
         taxable_ss      total_ss            agi    net_income  \
9975   28365.751953  33371.472656  125058.929688   7752.429688   
7144   20838.447266  40252.449219   53828.378906  35324.628906   
14142  15610.443359  34656.933594  115057.203125  23233.453125   
8523   14997.000000  17643.529297   98062.640625  27958.996094   
18336  14997.000000  29926.786133   53946.662354  33065.968750   
13276   2442.788574   4885.577148   31122.537109  35294.285156   
8800    1680.335938   3360.671875   32137.298828  36858.941406   
2495    1596.329102   5826.565430   26527.355469  36932.503906   
7559    1482.435547   7599.319824   25647.298828  36688.480469   
8592    1336.018555   3335.467041   27339.277344  32316.238281   
17641   1260.251953   2520.503906   33647.609375  34695.062500   
12862   1085.500000   1365.273071   34876.820312  36611.191406   
8117    1050.210083   2100.420166   27267.9238

In [24]:
# ============== Q1 UNITS WITH CHANGE IN NET INCOME ==============
print("\n" + "=" * 60)
print("Q1 HOUSEHOLDS WITH CHANGE IN NET INCOME FROM REFORM")
print("=" * 60)

# Filter for Q1 households with non-zero change
q1_mask_hh = quintiles_hh == 1
has_change_mask = np.abs(change_in_net_income.values) > 0.01
combined_q1_change_mask = q1_mask_hh & has_change_mask

baseline_taxable_ss = hh_vars['taxable_social_security']
reformed_taxable_ss = reformed.calculate("taxable_social_security", PERIOD, map_to="household")
benefits = baseline.calculate("household_benefits", PERIOD, map_to="household")
taxes = baseline.calculate("household_tax_before_refundable_credits", PERIOD, map_to="household")
market_income = baseline.calculate("household_market_income", PERIOD, map_to="household")
household_refundable_tax_credits = baseline.calculate("household_refundable_tax_credits", PERIOD, map_to="household")
gross_income = baseline.calculate("irs_gross_income", PERIOD, map_to="household")

df_q1_changes = pd.DataFrame({
    'baseline_net_income': baseline_income.values[combined_q1_change_mask],
    'market_income': market_income.values[combined_q1_change_mask],
    'benefits': benefits.values[combined_q1_change_mask],
    'taxes_before_refundable_credits': taxes.values[combined_q1_change_mask],
    'gross_income': gross_income.values[combined_q1_change_mask],
    'agi': hh_vars['adjusted_gross_income'].values[combined_q1_change_mask],
    'combined_income_for_ss_tax': hh_vars['combined_income_for_ss_tax'].values[combined_q1_change_mask],
    'reformed_net_income': reformed_income.values[combined_q1_change_mask],
    'change_in_net_income': change_in_net_income.values[combined_q1_change_mask],
    'baseline_taxable_ss': baseline_taxable_ss.values[combined_q1_change_mask],
    'social_security': hh_vars['social_security'].values[combined_q1_change_mask],
    'weight': baseline_income.weights[combined_q1_change_mask],
    
})

print(f"\nFound {combined_q1_change_mask.sum()} Q1 households with change in net income")
print(f"Total weighted Q1 households affected: {df_q1_changes['weight'].sum():,.0f}")

avg_change_q1 = np.average(df_q1_changes['change_in_net_income'], weights=df_q1_changes['weight'])
print(f"Average change in Q1 (affected households): ${avg_change_q1:,.2f}")

print(f"\nTop 20 Q1 households by change in net income:")
print(df_q1_changes.sort_values('change_in_net_income', ascending=False).head(20))


Q1 HOUSEHOLDS WITH CHANGE IN NET INCOME FROM REFORM

Found 14 Q1 households with change in net income
Total weighted Q1 households affected: 270,074
Average change in Q1 (affected households): $1,997.08

Top 20 Q1 households by change in net income:
       baseline_net_income  market_income      benefits  \
8523          27958.996094   23762.824219  17973.529297   
9975           7752.429688   -3644.802002  33371.472656   
7144          35324.628906   -5540.452148  44178.488281   
18336         33065.968750    7022.522949  29926.785156   
13276         35294.285156   33565.324219   4885.577148   
8800          36858.941406   37162.019531   3360.671875   
7559          36688.480469   33601.445312   7599.319824   
8592          32316.238281   29339.769531   3335.467041   
17641         34695.062500   35821.511719   2520.503906   
12862         36611.191406   38658.210938   1365.273071   
8117          29571.214844   28318.132812   2100.420166   
13279         33656.945312   36218.273438